Neste arquivo, serão utilizadas as bibliotecas Pandas e Numpy para prática:

In [1]:
import pandas as pd 
import numpy as np

Converta os países do dataframe abaixo para variáveis dummies eliminando a primeira coluna e utilizando 'Country' como prefixo no nome das colunas.

In [4]:
ids = [11, 22, 33, 44, 55, 66, 77]
countries = ['Brasil', 'Argentina', 'Peru', 'Bolívia', 'Uruguai']
sales = [42, 53, 37, 45, 20]

df = pd.DataFrame(list(zip(ids, countries, sales)),
                  columns=['Ids', 'Countries', 'Sales'])
df

,Ids,Countries,Sales
0,11,Brasil,42
1,22,Argentina,53
2,33,Peru,37
3,44,Bolívia,45
4,55,Uruguai,20


In [7]:
pd.get_dummies(df,prefix='Country',prefix_sep=' ',columns=['Countries'],drop_first=True)

,Ids,Sales,Country Bolívia,Country Brasil,Country Peru,Country Uruguai
0,11,42,0,1,0,0
1,22,53,0,0,0,0
2,33,37,0,0,1,0
3,44,45,1,0,0,0
4,55,20,0,0,0,1


Para os próximos passos, será considerado o dataset disponível em https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data que traz a especificação de modelos de carros em 1985:


In [46]:
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]
cars=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data',names=headers,header=0)
cars.replace('?',np.NaN,inplace=True)

Esse dataset possui NaNs? Se sim, trate-os como julgar melhor.

In [49]:
cars.isna().sum()

symboling             0
normalized_losses    40
make                  0
fuel_type             0
aspiration            0
num_doors             2
body_style            0
drive_wheels          0
engine_location       0
wheel_base            0
length                0
width                 0
height                0
curb_weight           0
engine_type           0
num_cylinders         0
engine_size           0
fuel_system           0
bore                  4
stroke                4
compression_ratio     0
horsepower            2
peak_rpm              2
city_mpg              0
highway_mpg           0
price                 4
dtype: int64

In [50]:
cars[['bore', 'stroke','normalized_losses','price']] = cars[['bore', 'stroke','normalized_losses','price']].astype('float64')
cars['bore'].fillna(cars.bore.mean(),inplace=True)
cars['stroke'].fillna(cars.stroke.mean(),inplace=True)
cars['price'].fillna(cars.price.median(),inplace=True)
cars['num_doors'].fillna('four',inplace=True)
cars.dropna(subset=['horsepower','peak_rpm'],inplace=True)
cars.normalized_losses.interpolate(method="polynomial", order=2,inplace=True)
cars.dropna(inplace=True)

    Os valores nulos para as colunas 'bore' e 'stroke' são preenchidas com a média, por haver um range muito pequeno entre todos os dados disponíveis. Os valores da coluna 'price' serão substituídos pela mediana, para desconsiderar o efeito dos outliers. Os valores da coluna 'num_doors' são preenchidos com o valor 'four', a moda desta coluna. Os valores de 'normalized_losses' são interpolados pelo método polinomial de ordem 2. Por fim, os valores nulos de 'horsepower' e 'peak_rpm' são dropados, por se tratar de poucos dados e com uma difícil determinação.

Esse dataset possui linhas duplicadas?

In [52]:
cars.duplicated().sum()

0

O consumo dos automóveis na cidade e rodovia, representados pelas colunas "city-mpg" e "highway-mpg" respectivamente, estão na unidade mpg (miles per gallon). Converta essas colunas para consumo em L/100km sabendo que a conversão é dada por:

L/100km = 235 / mpg 

In [55]:
cars['city_mpg']=235/cars['city_mpg']
cars['highway_mpg']=235/cars['highway_mpg']

Converta as colunas "body_style" e "drive_wheels" para variáveis dummies.

In [59]:
pd.get_dummies(cars,columns=['drive_wheels','body_style'],drop_first=True)

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,engine_location,wheel_base,length,width,...,peak_rpm,city_mpg,highway_mpg,price,drive_wheels_fwd,drive_wheels_rwd,body_style_hardtop,body_style_hatchback,body_style_sedan,body_style_wagon
2,2,164.000000,audi,gas,std,four,front,99.8,176.6,66.2,...,5500,9.791667,7.833333,13950.0,1,0,0,0,1,0
3,2,164.000000,audi,gas,std,four,front,99.4,176.6,66.4,...,5500,13.055556,10.681818,17450.0,0,0,0,0,1,0
4,2,161.597066,audi,gas,std,two,front,99.8,177.3,66.3,...,5500,12.368421,9.400000,15250.0,1,0,0,0,1,0
5,1,158.000000,audi,gas,std,four,front,105.8,192.7,71.4,...,5500,12.368421,9.400000,17710.0,1,0,0,0,1,0
6,1,154.417606,audi,gas,std,four,front,105.8,192.7,71.4,...,5500,12.368421,9.400000,18920.0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,-1,95.000000,volvo,gas,std,four,front,109.1,188.8,68.9,...,5400,10.217391,8.392857,16845.0,0,1,0,0,1,0
200,-1,95.000000,volvo,gas,turbo,four,front,109.1,188.8,68.8,...,5300,12.368421,9.400000,19045.0,0,1,0,0,1,0
201,-1,95.000000,volvo,gas,std,four,front,109.1,188.8,68.9,...,5500,13.055556,10.217391,21485.0,0,1,0,0,1,0
202,-1,95.000000,volvo,diesel,turbo,four,front,109.1,188.8,68.9,...,4800,9.038462,8.703704,22470.0,0,1,0,0,1,0
